# Building symbolic metamodels

A symbolic metamodel takes as an input a machine learning model, and outputs a symbolic equation describing its response surface as illustrated in the Figure below. This notebook provides the steps needed for building a symbolic metamodel for an XGBoost model fitted to the "UCI absenteeism at work dataset: https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

<img src="images/FigB1.png", width=640>

We first import the necessary modules for symbolic metamodeling

In [1]:
from pysymbolic.algorithms.symbolic_metamodeling import *

Next, we import the necessary libraries fordata processing, splitting the data into training and testing samples, and XGBoost model fitting 

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


data          = pd.read_csv("data/absenteeism.csv", delimiter=';')

feature_names = ['Transportation expense', 'Distance from Residence to Work',
                 'Service time', 'Age', 'Work load Average/day ', 'Hit target',
                 'Disciplinary failure', 'Education', 'Son', 'Social drinker',
                 'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index']

scaler        = MinMaxScaler(feature_range=(0, 1))
X             = scaler.fit_transform(data[feature_names])
Y             = ((data['Absenteeism time in hours'] > 4) * 1) 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

model         = RandomForestClassifier()

model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
Y

0      0
1      0
2      0
3      0
4      0
      ..
735    1
736    0
737    0
738    0
739    0
Name: Absenteeism time in hours, Length: 740, dtype: int64

Let us examine the AUC-ROC performance of the fitted XGBoost model on the test data

In [14]:
roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])

0.6977779386218876

How much better does XGBoost perform compared to a standard logistic regression?

In [15]:
model_L = LogisticRegression()

model_L.fit(X_train, Y_train)

roc_auc_score(Y_test, model_L.predict_proba(X_test)[:, 1])

0.6704183555298204

Now let us create a symbolic metamodel instance. This takes the fitted model **model** and the training features **X_train** as follows

In [17]:
metamodel = symbolic_metamodel(model, X_train)

metamodel.fit(num_iter=2, batch_size=X_train.shape[0], learning_rate=0.1) #10, 0.01

---- Tuning the basis functions ----


KeyboardInterrupt: 

Now let us see how this metamodel performs using the **evaluate** method...

In [ ]:
Y_metamodel = metamodel.evaluate(X_test)

roc_auc_score(Y_test, Y_metamodel)

It performs close to the original XGBoost model! Now let us see the exact symbolic equation of the model in terms of Meijer-G functions

By invoking the **.fit()** method in the **symbolic_metamodel** class, we essentially transform the XGBoost model to a space of interpretable symbolic equations as shown in the Figure below, without much loss in predictive accuracy. 

<img src="images/FigB2.png", width=320>

Now we show how to extract the exact and approximate equation $g(x)$ from the metamodel class...

In [ ]:
metamodel.exact_expression

Because this equation involves Hypergeometric functions, we might prefer to work with the polynomial approximation...

In [ ]:
metamodel.approx_expression